<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

# RNA in PyRosetta
Keywords: classify_base_pairs, RNA torsions, RNA score terms, RNA motifs, mutate_position, RNA thread

In this lab, we will explore common tasks and approaches for working with RNA using Rosetta. We will be focusing on a simple system that includes a helix capped by a tetraloop for this exercise.

In [1]:
# Notebook setup
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.setup()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

In [2]:
from pyrosetta import *
init()

PyRosetta-4 2019 [Rosetta PyRosetta4.Release.python36.ubuntu 2019.44+release.5aed75f2e796a33ab71515b6c1daa321eb2294a2 2019-10-29T08:37:43] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python36.ubuntu r237 2019.44+release.5aed75f2e79 5aed75f2e796a33ab71515b6c1daa321eb2294a2 http://www.pyrosetta.org 2019-10-29T08:37:43
core.init: command: PyRosetta -ex1 -ex2aro -database /usr/local/lib/python3.6/dist-packages/pyrosetta-2019.44+release.5aed75f2e79-py3.6-linux-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-1940380485 seed_offset=0 real_seed=-1940380485
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-1940380485 RG_type=mt19937


In [3]:
from pyrosetta.rosetta import *
from pyrosetta.rosetta.core.pose.rna import *
from pyrosetta.rosetta.core.pose import *

## Exploring geometry for RNA ##

Let's load in this structure with PyRosetta (make sure that you have the PDB file located in your current directory):

`cd google_drive/My\ Drive/student-notebooks/
pose = pose_from_pdb("inputs/stem_loop.pdb")`

In [4]:
### BEGIN SOLUTION
pose = pose_from_pdb("inputs/stem_loop.pdb")
### END SOLUTION

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 980 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 1.35938 seconds.
core.import_pose.import_pose: File 'inputs/stem_loop.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.chirality_resolution: Flipping atom xyz for H5' and H5'' for residue   C


Let's explore the structure in this PDB file. First, use `pose.sequence()` to look at the sequence:

In [5]:
# print out the sequence of the pose
### BEGIN SOLUTION
pose.sequence()
### END SOLUTION

'cauccgaaaggaug'

We can see that the pose seems to contain RNA residues. To check this, let's go through the pose residue by residue, checking if each one is RNA.

In [6]:
for ii in range(pose.size()):
    print(pose.residue_type(5).is_RNA())

True
True
True
True
True
True
True
True
True
True
True
True
True
True


RNA bases interact with each other via **base pairing**, either through the Watson-Crick base pairs that make up standard A-form helices or through non-canonical base pairing interactions. We can use the `classify_base_pairs` function (this lives in `core:pose:rna` which was loaded above) to find and classify all the base pairs in the current pose. Let's take a look.

In [7]:
base_pairs = classify_base_pairs(pose)
for base_pair in base_pairs:
    print(base_pair)

basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/AccStrength.csv
1 14 WC WC ANTI
2 13 WC WC ANTI
3 12 WC WC ANTI
4 11 WC WC ANTI
5 10 WC WC ANTI
6 9 SUGAR HOOG ANTI


We can see that the RNA molecule consists of Watson-Crick base pairs between residues 1-5 and residues 10-14 forming a standard RNA helix. We can also see that residues 6 and 9 form a non-canonical base pair interaction between the sugar and Hoogsteen edges of the respective bases. We can think of this structure as a simple stem-loop, with an idealized A-form helix between residues 1-5 and residues 10-14, and with a tetraloop connecting these chains.

Let's use some of Rosetta's tools for measuring **distances and torsions** to understand the typical geometry of an idealized A-form helix.

What is the distance between the phosphate atoms of two consecutive residues in one strand of a helix? Check this for a couple pairs of residues.

In [8]:
P1_xyz = pose.residue(1).xyz("P")
P2_xyz = pose.residue(2).xyz("P")
P3_xyz = pose.residue(3).xyz("P")
print((P1_xyz - P2_xyz).norm())
print((P2_xyz - P3_xyz).norm())

5.923353746008057
5.846964169549871


RNA nucleotides are quite large compared to amino acids, with many more torsion angles. In the diagram of a nucleotide below, we can see the backbone torsions applicable to RNA: $\alpha$, $\beta$, $\gamma$, $\delta$, $\epsilon$, $\zeta$, and $\chi$.

![nucleotide_torsions](Media/nucleotide_torsions.png)

We can access the values of these torsions through the pose object. Just like protein torsions can be accessed with functions like `pose.phi(resid)`, RNA torsions can be accessed with analogous functions like `pose.alpha(resid)`.

**Exercise**: Below, make a function that prints out all the torsions for a given residue. Then, using that function, check the torsions for three different residues in the RNA helix. How similar are torsion angles for different residues in an idealized helix?

In [9]:
def print_torsions(pose, resi):
    print("%s: %f" % ("alpha", pose.alpha(resi)))
    print("%s: %f" % ("beta", pose.beta(resi)))
    print("%s: %f" % ("gamma", pose.gamma(resi)))
    print("%s: %f" % ("delta", pose.delta(resi)))
    print("%s: %f" % ("epsilon", pose.epsilon(resi)))
    print("%s: %f" % ("zeta", pose.zeta(resi)))
    print("%s: %f" % ("chi", pose.chi(resi)))

In [10]:
print("Torsions for residue 2:")
print_torsions(pose, 2)
print("Torsions for residue 3:")
print_torsions(pose, 3)
print("Torsions for residue 12:")
print_torsions(pose, 12)

Torsions for residue 2:
alpha: -72.516450
beta: 178.544461
gamma: 58.327244
delta: 79.813633
epsilon: -152.375244
zeta: -70.561778
chi: 74.780524
Torsions for residue 3:
alpha: -65.899308
beta: 173.462004
gamma: 56.858161
delta: 79.609718
epsilon: -152.592269
zeta: -70.540365
chi: 77.981938
Torsions for residue 12:
alpha: -67.502850
beta: 174.408225
gamma: 56.171977
delta: 81.812182
epsilon: -149.905716
zeta: -76.028946
chi: 75.291052


## Scoring RNA poses ##

Rosetta's energy functions provide a mechanism to score RNA structures, rewarding realistic conformations using a variety of score terms. In this section, we will see how to score RNA poses, and we will use these score terms to better understand our structure.

To score structures with RNA in Rosetta, it is best to use a high-resolution energy function designed to work with RNA, for instance `stepwise/rna/rna_res_level_energy4.wts`. In fact, the standard high resolution energy function used in Rosetta does not include the score terms that are quite helpful for modeling RNA. To see this, we will evaluate our RNA pose with the `ref2015` score function and `stepwise/rna/rna_res_level_energy4.wts`, comparing the resulting score term values.

In [35]:
hires_sf = core.scoring.ScoreFunctionFactory.create_score_function("ref2015");

core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/rama/fd/all.ramaProb
basic.io.database: Database file opened: scoring/score_functions/rama/fd/prepro.ramaProb
basic.io.database: Database file opened: scoring/score_functions/omega/omega_ppdep.all.txt
basic.io.database: Database file opened: scoring/score_functions/omega/omega_ppdep.gly.txt
basic.io.database: Database file opened: scoring/score_functions/omega/omega_ppdep.pro.txt
basic.io.database: Database file opened: scoring/score_functions/omega/omega_ppdep.valile.txt
basic.io.database: Database file opened: scoring/score_functions/P_AA_pp/P_AA
basic.io.database: Dat

In [36]:
hires_sf.show(pose)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000    -143.548    -143.548
 fa_rep                       0.550      25.767      14.172
 fa_sol                       1.000     178.133     178.133
 fa_intra_rep                 0.005     128.623       0.643
 fa_intra_sol_xover4          1.000      49.210      49.210
 lk_ball_wtd                  1.000     -19.646     -19.646
 fa_elec                      1.000      -4.847      -4.847
 pro_close                    1.250       0.000       0.000
 hbond_sr_bb                  1.000       0.000       0.000
 hbond_lr_bb                  1.000       0.000       0.000
 hbond_bb_sc                  1.000      -0.643      -0.643
 hbond_sc                     1.000      -8.489      -8.489
 dslf_fa13                    1.250       0.000       0.000
 omega  

Note that `ref2015` does contain some terms that are used for RNA modeling like VDW and hydrogen bonding score terms. What extra terms are included in the RNA high resolution score function?

In [37]:
rna_hires_sf = core.scoring.ScoreFunctionFactory.create_score_function("stepwise/rna/rna_res_level_energy4.wts");

In [38]:
rna_hires_sf.show(pose)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       0.210    -212.486     -44.622
 fa_rep                       0.200      36.392       7.278
 fa_intra_rep                 0.003     211.801       0.614
 lk_nonpolar                  0.250      -5.706      -1.427
 fa_elec_rna_phos_phos        1.700      -1.064      -1.810
 rna_torsion                  1.000       5.682       5.682
 suiteness_bonus              1.000       0.000       0.000
 rna_sugar_close              0.820       0.770       0.631
 fa_stack                     0.130    -226.828     -29.488
 stack_elec                   0.760      -5.919      -4.499
 geom_sol_fast                0.170      61.291      10.419
 hbond_sr_bb_sc               0.960      -0.643      -0.617
 hbond_lr_bb_sc               0.960       0.000       0.000
 hbond_s

We can see that some new score terms in the high resolution RNA potential, including `rna_torsion`, `suiteness_bonus`, `rna_sugar_close`, and `fa_stack`. We will explore a few of these terms below. To learn about these and other score terms that have been included to more realistically model RNA, check out these papers: 

Analogous to the protein low-resolution potential, an RNA low-resolution potential has been developed to more quickly score RNA structures represented in centroid mode. Lets take a look at the score terms involved.

In [39]:
rna_lowres_sf = core.scoring.ScoreFunctionFactory.create_score_function("rna/denovo/rna_lores_with_rnp_aug.wts");

In [40]:
rna_lowres_sf.show(pose)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 rna_data_backbone            2.000       0.000       0.000
 rna_vdw                      2.000       1.753       3.506
 rnp_vdw                     50.000       0.000       0.000
 rna_base_backbone            2.000      -5.980     -11.961
 rna_backbone_backbone        2.000       0.000       0.000
 rna_repulsive                5.000       0.061       0.304
 rna_base_pair                2.000     -28.186     -56.372
 rna_base_axis                0.400      -9.351      -3.740
 rna_base_stagger             2.000      -4.044      -8.087
 rna_base_stack               2.000       0.000       0.000
 rna_base_stack_axis          0.400       0.000       0.000
 rnp_base_pair                2.000       0.000       0.000
 rnp_stack                    5.000       0.000       0.000
 rnp_pai

We can see that when modeling an RNA molecule using centroid positions for nucleotides, we need to separately include terms for base pairing (`rna_base_pair`), base-backbone interactions (`rna_base_backbone`), and so on. 

Returning to the high resolution RNA score function, let us see if we can decompose the energies further to understand which parts of the structure contribute positively and negatively to its energy. First, we can decompose the energies per residue like below.

In [41]:
rna_hires_sf(pose)
nonzero_scores = pose.energies().residue_total_energies(4).show_nonzero()
print(nonzero_scores)

 fa_atr: -14.4023 fa_rep: 4.67417 fa_intra_atr: -10.5282 fa_intra_rep: 13.0994 fa_intra_sol: 6.12965 fa_intra_atr_nonprotein: -10.5282 fa_intra_rep_nonprotein: 13.0994 fa_intra_sol_nonprotein: 6.12965 lk_nonpolar: -0.607911 fa_elec_rna_phos_phos: -0.097892 rna_torsion: 0.789858 rna_torsion_sc: 0.699603 rna_sugar_close: 0.033771 fa_stack: -12.9687 stack_elec: -1.05115 geom_sol_fast: 4.17629 geom_sol_fast_intra_RNA: 0.429881 hbond_sr_bb_sc: -0.321401 hbond_sc: -0.0773844 ref: 2.82 total_score: -0.890184


A lot of the RNA specific energy terms make more sense when we look at pairs of residues. The energy graph object allows you to explore pairwise energies. The function below uses the energy graph to print out all non-zero scores between residues for a particular score term.

In [42]:
def print_nonzero_pairwise_energies(pose, energy_term, sf):
    sf(pose)
    energy_graph = pose.energies().energy_graph()
    for ii in range(1, pose.size() + 1):
        for jj in range(ii + 1, pose.size() + 1):
            edge = energy_graph.find_energy_edge(ii, jj)
            if (edge != None):
                emap = edge.fill_energy_map()
                resid1 = str(ii) + " " + pose.residue(ii).name1()
                resid2 = str(jj) + " " + pose.residue(jj).name1()
                resid_pair = resid1 + " " + resid2
                score = emap[ energy_term ]
                if score != 0:
                    print("%s: %f" % (resid_pair, score))

Using the function above, we're going to look at the stacking energies in the high resolution potential.

In [43]:
print_nonzero_pairwise_energies(pose, core.scoring.ScoreType.fa_stack, rna_hires_sf)

1 c 2 a: -16.451113
1 c 3 u: -0.009178
1 c 12 a: -0.607248
1 c 13 u: -0.994826
2 a 3 u: -17.461226
2 a 4 c: -0.080392
2 a 11 g: -1.163378
2 a 12 a: -0.516092
3 u 4 c: -14.451225
3 u 5 c: -0.053151
3 u 6 g: -0.359144
3 u 11 g: -0.441683
3 u 12 a: -0.265338
4 c 5 c: -6.123487
4 c 6 g: -3.780118
4 c 10 g: -0.152583
4 c 11 g: -1.349600
5 c 6 g: -11.242745
5 c 8 a: -0.126661
5 c 9 a: -1.159588
5 c 10 g: -1.778713
5 c 11 g: -1.445591
6 g 8 a: -0.448459
6 g 9 a: -1.665336
6 g 10 g: -3.780769
6 g 11 g: -2.547230
7 a 8 a: -20.368649
8 a 9 a: -20.065970
9 a 10 g: -22.016733
10 g 11 g: -19.865254
11 g 12 a: -20.601959
12 a 13 u: -18.468119
12 a 14 g: -0.022942
13 u 14 g: -16.963790


We can see that the **stacking energies** are highest for consecutive residues. In the idealized helix, the best stacking energy bonuses are given to stacked purines.

Now lets take a look at the **torsion energies**. Which energies are the highest? Where are these torsions in the structure?

In [44]:
print_nonzero_pairwise_energies(pose, core.scoring.ScoreType.rna_torsion, rna_hires_sf)

1 c 2 a: 0.126091
2 a 3 u: 0.038852
3 u 4 c: 0.019435
4 c 5 c: 0.122484
5 c 6 g: 0.351609
6 g 7 a: 0.468183
7 a 8 a: 0.063458
8 a 9 a: 0.011509
9 a 10 g: 0.064941
10 g 11 g: 0.085545
11 g 12 a: 0.084511
12 a 13 u: 0.055541
13 u 14 g: 0.021543


RNA structures are often viewed as being composed of small building blocks called **RNA motifs**. These motifs can be as simple as stacks of base pairs, which we have seen above. Typical motifs also include stereotyped loops, junctions, and tertiary contacts present across many common RNA molecules. Let's take a look to see whether any of these common RNA motifs are present in our simple stem loop structure.

In [123]:
lowres_potential = core.scoring.rna.RNA_LowResolutionPotential( "scoring/rna/rna_base_pair_xy.dat" )
rna_scoring_info = core.scoring.rna.rna_scoring_info_from_pose(pose).rna_filtered_base_base_info()
rna_motifs = core.scoring.rna.get_rna_motifs( pose, lowres_potential, rna_scoring_info)
print(rna_motifs)

core.scoring.rna.RNA_LowResolutionPotential: Reading basepair x - y potential file: scoring/rna/rna_base_pair_xy.dat
core.scoring.rna.RNA_LowResolutionPotential: Finished reading basepair x - y potential file: scoring/rna/rna_base_pair_xy.dat
core.scoring.rna.RNA_LowResolutionPotential: Reading non - base - base x - y potential file: scoring/rna/rna_base_backbone_xy.dat
core.scoring.rna.RNA_LowResolutionPotential: Reading RNA backbone backbone potential file: scoring/rna/rna_backbone_backbone.dat
WC_STACKED_PAIR [4, 5, 10, 11]
WC_STACKED_PAIR [11, 12, 3, 4]
WC_STACKED_PAIR [10, 11, 4, 5]
WC_STACKED_PAIR [3, 4, 11, 12]
U_TURN [6, 7, 8]



We can see that our RNA structure includes many stacked Watson-Crick base pair, making the idealized A-form helix. In addition, the loop connecting the strands of the helix in our structure is a stereotyped "GNRA" tetraloop, taking a loop conformation that is common across many RNA structures in the PDB.

## Manipulating RNA poses ##

Rosetta allows you to not just explore a given PDB structure, but to manipulate and design structures. In this section, we discuss some basic ways to manipulate RNA structures, and we observe the effects of these manipulations on the structure's energy. For each manipulation, we will make a new copy of the pose to make sure that our changes do not affect the original structure we loaded in.

One basic manipulation we can make to an RNA structure is to change torsion angles for individual residues. Let's try this out on a residue in the A-form helix, and observe the effect on the rna_torsion score. Did the change we made make the score better or worse?

In [22]:
new_pose = Pose()
new_pose.assign(pose)
rna_hires_sf(pose)
torsion_score_before = pose.energies().total_energies()[core.scoring.ScoreType.rna_torsion]
new_pose.set_beta(2, 110)
rna_hires_sf(new_pose)
torsion_score_after = new_pose.energies().total_energies()[core.scoring.ScoreType.rna_torsion]
print("%s: %f" % ("Torsion score before", torsion_score_before))
print("%s: %f" % ("Torsion score after", torsion_score_after))

Torsion score before: 3.750853
Torsion score after: 4.166565


If you want to replace residues in an RNA molecule with their idealized versions, you can use the RNA_IdealCoord class in Rosetta. Below is an example for using that method to first replace a single residue with its idealized version, and then to replace all residues with their idealized versions across the whole pose.

In [23]:
ideal_pose_one = Pose()
ideal_pose_one.assign(pose)
resid = 2
core.pose.rna.RNA_IdealCoord().apply(ideal_pose_one, resid, core.chemical.rna.PuckerState.ANY_PUCKER, False)

ideal_pose = Pose()
ideal_pose.assign(pose)
core.pose.rna.RNA_IdealCoord().apply(ideal_pose, False)

**Exercise**: Figure out if the total energy of the pose went up or down after replacing one or all of the residues with their idealized versions. What can explain the difference?

In [24]:
rna_hires_sf(pose)
rna_hires_sf(ideal_pose)
rna_hires_sf(ideal_pose_one)
print(pose.energies().total_energy())
print(ideal_pose.energies().total_energy())
print(ideal_pose_one.energies().total_energy())

-18.045673691089746
22.634803255900167
-1.1025695607202337


Another common manipulation for an RNA structure is to mutate the nucleotides to different bases. This is a manipulation that is commonly used while modeling one RNA structure using coordinates from another homologous (but not identical) structure. Below we can see how to mutate one residue of our RNA structure to another one. We include the `update_full_model_info_from_pose` command to ensure that the pose's internal data is kept up-to-date after the mutation is made.

In [25]:
mutated_pose = Pose()
mutated_pose.assign(pose)
print(pose.sequence())
rosetta.core.pose.rna.mutate_position(mutated_pose, 1, 'a')
core.pose.full_model_info.update_full_model_info_from_pose(mutated_pose)
print(mutated_pose.sequence())

cauccgaaaggaug
aauccgaaaggaug


**Exercise**: Make a function that mimics the 'rna_thread' Rosetta application, which takes in a pose and a new sequence and replaces all pose residues with the new sequence's residues. Remember to check that the pose's sequence and the new sequence have the same length. 

The pose's current sequence is `cauccgaaaggaug`. Use the function you wrote to make a version that has sequence `cauccuucgggaug` and one that has sequence `aaaaagaaauuuuu`.

In [32]:
def rna_thread(pose, new_seq):
    if len(pose.sequence()) != len(new_seq):
        print("Sequences have different length; cannot rethread")
        return pose
    for ii in range(1, pose.size()+1):
        rosetta.core.pose.rna.mutate_position(pose, ii, new_seq[ii-1])
        core.pose.full_model_info.update_full_model_info_from_pose(pose)
    return pose   

In [33]:
uucg_pose = Pose()
uucg_pose.assign(pose)
uucg_pose = rna_thread(uucg_pose, 'cauccuucgggaug')
print(uucg_pose.sequence())

auhelix_pose = Pose()
auhelix_pose.assign(pose)
auhelix_pose = rna_thread(auhelix_pose, 'aaaaagaaauuuuu')
print(auhelix_pose.sequence())

cauccuucgggaug
aaaaagaaauuuuu


**Exercise**: The RNA high resolution potential includes hydrogen bonding terms. CG base pairs have more hydrogen bonds than AU base pairs. Compare the original pose with the pose that has all AU base pairs. What happens to the hydrogen bonding energy in the high resolution potential? 

In [34]:
rna_hires_sf(auhelix_pose)
rna_hires_sf(pose)
auhelix_hbond_sc = auhelix_pose.energies().total_energies()[core.scoring.ScoreType.hbond_sc]
pose_hbond_sc = pose.energies().total_energies()[core.scoring.ScoreType.hbond_sc]
print('%s: %f' % ("AU Helix", auhelix_hbond_sc))
print('%s: %f' % ("CG Helix", pose_hbond_sc))

NameError: name 'rna_hires_sf' is not defined

**Exercise**: The stacking energies of the GAAA and UUCG tetraloops differ from each other. Which tetraloop provides the most favorable stacking energies overall? Can you figure out which pairs of residues have different stacking energies when the structure has changed? 

In [29]:
rna_hires_sf(uucg_pose)
rna_hires_sf(pose)
uucg_fa_stack = uucg_pose.energies().total_energies()[core.scoring.ScoreType.fa_stack]
pose_fa_stack = pose.energies().total_energies()[core.scoring.ScoreType.fa_stack]
print('%s: %f' % ("UUCG Loop", uucg_fa_stack))
print('%s: %f' % ("GAAA Loop", pose_fa_stack))

UUCG Loop: -214.060807
GAAA Loop: -234.442137


In [30]:
rna_hires_sf(uucg_pose)
rna_hires_sf(pose)
pose_energy_graph = pose.energies().energy_graph()
uucg_energy_graph = uucg_pose.energies().energy_graph()
for ii in range(1, pose.size() + 1):
    for jj in range(ii + 1, pose.size() + 1):
        pose_edge = pose_energy_graph.find_energy_edge(ii, jj)
        uucg_edge = uucg_energy_graph.find_energy_edge(ii, jj)
        if (pose_edge != None)  and (uucg_edge != None):
            pose_emap = pose_edge.fill_energy_map()
            uucg_emap = uucg_edge.fill_energy_map()
            resid1 = str(ii) + " " + pose.residue(ii).name1()
            resid1_uucg = str(ii) + " " + uucg_pose.residue(ii).name1()
            resid2 = str(jj) + " " + pose.residue(jj).name1() 
            resid2_uucg = str(jj) + " " + uucg_pose.residue(jj).name1()
            resid_pair = resid1 + ", " + resid2
            resid_pair_uucg = resid1_uucg + ", " + resid2_uucg
            pose_score = pose_emap[ core.scoring.ScoreType.fa_stack ]
            uucg_score = uucg_emap[ core.scoring.ScoreType.fa_stack ]
            if pose_score != uucg_score:
                print("%s: %f; %s: %f" % (resid_pair, pose_score, resid_pair_uucg, uucg_score))

4 c, 6 g: -0.042999; 4 c, 6 u: -0.020146
5 c, 6 g: -14.998700; 5 c, 6 u: -14.424303
5 c, 9 a: -0.055888; 5 c, 9 g: -0.040952
6 g, 8 a: -1.056694; 6 u, 8 c: -0.798130
6 g, 9 a: -0.381851; 6 u, 9 g: -0.142689
6 g, 10 g: -11.820445; 6 u, 10 g: -5.138552
7 a, 8 a: -21.526270; 7 u, 8 c: -14.881668
8 a, 9 a: -22.045880; 8 c, 9 g: -16.479345
9 a, 10 g: -13.441684; 9 g, 10 g: -13.063297


## Elements of RNA Structure Prediction

Many of the same strategies are used when modeling RNA as when modeling proteins. Below, we shall explore some of these procedures specifically applied to RNA molecules to appreciate how they may come together to give a modern structure prediction method.

### Generating an ideal A-form Helix

On a not wholly unrelated tangent, let us first see how we can quickly generate poses of ideal A-form RNA. You can think of this procedure as analogous to the `pose_from_seq` function used to generate protein poses from primary sequences. Let's use it now to generate a single-strand RNA pose with A-form torsions and the same sequence as the hairpin we've been examining so far.

```python
assembler = core.import_pose.RNA_HelixAssembler()
assembled_pose = assembler.build_init_pose(pose.sequence(), '')
```



In [127]:
### BEGIN SOLUTION
assembler = core.import_pose.RNA_HelixAssembler()
assembled_pose = assembler.build_init_pose(pose.sequence(), '')
### END SOLUTION

Let's get a `PyMOLMover` up and running so we can examine our new pose.

In [128]:
pmm = PyMOLMover()
pmm.apply(assembled_pose)

You can also use `RNA_HelixAssmebler` to generate poses that comprise two strands that form an ideal A-form helical stack, like residues 1-5 and 10-14 in the hairpin from above.

In [129]:
helix_pose = assembler.build_init_pose('ggg','ccc')

Looking in PyMOL, you may be able to appreciate that, true to its name, the `RNA_HelixAssembler` has generated a pose that looks quite helical.

In [130]:
pmm.apply(helix_pose)

**Exercise**: Examine the torsions in several of the residues of `assembled_pose` using the `print_torsions` function you wrote earlier. How do they compare to the torsions from the starting stem loop?

### RNA Fragments

Given a library of RNA torsions excised from a published structure, fragment assembly methods will choose an n-mer in the current structure and replace the backbone geometry with the geometry of a corresponding n-mer from the library. Those of you familiar with protein structure prediction methods will recognize this stragety of fragment assembly.

We will implement a rudimentary version of this protocol for RNA below.

For the present exercise, we will use the torsions file `inputs/1jj2.torsions`, which comes from the crystal structure of a large ribosomal subunit. This library will be used to initialize a `Mover` specifically designed to perform fragment assembly on RNA molecules, `RNA_FragmentMover`.
```python
fragset = core.import_pose.libraries.RNA_LibraryManager.get_instance().rna_fragment_library("inputs/1jj2.torsions")
atom_level_domain_map = core.pose.toolbox.AtomLevelDomainMap(assembled_pose)
frag_mover = protocols.rna.denovo.movers.RNA_FragmentMover(fragset, atom_level_domain_map, 1, 0)
```
Don't worry too much about the other options that `RNA_FragmentMover`  requires at this point, but remember to include them if using this mover outside of this notebook.

In [131]:
### BEGIN SOLUTION
fragset = core.import_pose.libraries.RNA_LibraryManager.get_instance().rna_fragment_library("inputs/1jj2.torsions")
atom_level_domain_map = core.pose.toolbox.AtomLevelDomainMap(assembled_pose)
frag_mover = protocols.rna.denovo.movers.RNA_FragmentMover(fragset, atom_level_domain_map, 1, 0)
### END SOLUTION

Let's practice applying this mover to a `Pose`. To actually make a fragment assembly move, you can call the `random_fragment_insertion` method which requires two arguments:

1. An input `Pose`
2. The size of the fragment to substitute.

There is also an `apply()` method that can be called in a similar manner, but it simply calls `random_fragment_insertion()`, so the recommendation is to decrease overhead by calling `random_fragment_insertion()` where possible.

Let's pratice calling this method below.
```python
practice_pose = Pose()
practice_pose.assign(assembled_pose)
frag_mover.random_fragment_insertion(practice_pose, 3)
pmm.apply(practice_pose)
```


In [149]:
### BEGIN SOLUTION
practice_pose = Pose()
practice_pose.assign(assembled_pose)
frag_mover.random_fragment_insertion(practice_pose, 3)
pmm.apply(practice_pose)
### END SOLUTION

Now that we know how to set up a fragment assembly mover in PyRosetta, try the excise below to write a quick folding routine that uses a fragment assembly strategy to try and fold the hairpin sequence. 

**Exercise**: Fill in the function below such that `fragment_assembly`
* Accepts an input `Pose`, `RNA_FragmentMover`, fragment size to substitute (`frag_size`), and number of trials (`n_trials`).
* Uses by default the `rna_lowres_sf` energy function from earlier but allows the user to specify a different energy function, if desired
* Performs a fragment substitution and accepts the substitution subject to the Metropolis criterion (assume $kT = 1$)
* Returns the lowest-energy pose found.

\* See section 4.1 of these notebooks for a review on Monte Carlo algorithms, if desired. 

Then, apply it to our newly assembled pose using the following recipe:

1. Run `fragment_assembly` using 3 nucleotide fragments for 400 trials. 
2. Then, run `fragment_assembly` using 2 nucleotide fragments for 300 trials.
3. Finally, run `fragment_assembly` using 1 nucleotide fragments for 300 trials.

In [150]:
# This is a complete solution, but the idea is that students would fill in the bit between the two solutions lines.
# Not sure if this is feasible (let alone desireable from a pedagogical point of view) --Matt
import math
import random

def fragment_assembly(start_pose, frag_mover, frag_size, n_trials, sf=rna_lowres_sf):
    curr_pose = Pose()
    curr_pose.assign(start_pose)
    trial_pose = Pose()
    trial_pose.assign(curr_pose)
    opt_pose = Pose()
    opt_pose.assign(curr_pose)
    currE = newE = optE = sf(curr_pose)
    ### BEGIN SOLUTION
    for _ in range(n_trials):
        frag_mover.random_fragment_insertion(trial_pose, frag_size)
        newE = sf(trial_pose)
        if random.random() < math.exp(currE-newE):
            curr_pose.assign(trial_pose)
            currE = newE
            if currE < optE:
                optE = currE
                opt_pose.assign(curr_pose)
    ### END SOLUTION
    return curr_pose

frag_pose = fragment_assembly(assembled_pose, frag_mover, 3, 400)
frag_pose = fragment_assembly(frag_pose, frag_mover, 2, 300)
frag_pose = fragment_assembly(frag_pose, frag_mover, 1, 300)

Examine the fragment assembled `Pose` in PyMOL. Do you recognize any of the motifs from before?
```python
pmm.apply(frag_pose)
```


In [151]:
### BEGIN SOLUTION
pmm.apply(frag_pose)
### END SOLUTION

### Minimizing Structures with RNA

In principle, the standard `MinMover` that has been introduced previously in the context of minimizing purely protein structures can also be used to minimize poses with RNA (as long as the assigned energy function has score terms relevant to RNA and an appropriate `MoveMap` is provided).

However, as part of the `rna_denovo` protocol, Das and coworkers have developed a subroutine, `RNA_Minimize`, that is specifically geared toward handling minimization of poses with RNA, the use of which is detailed below.

TODO(?) (for Matt): Add demonstration of usual `MinMover` on RNA structure? This may be worth doing because need to be careful about how the `MoveMap` is set up etc.

We can access the `RNA_Minimize` mover from the `protocols` namespace. The relevant options object, `RNA_MinimizerOptions`, lives in the `import_pose.options` namespace. We will use the default options save setting the maximum number of iterations to 1000.
```python
rna_min_options = core.import_pose.options.RNA_MinimizerOptions()
rna_min_options.set_max_iter(1000)
rna_minmizer = protocols.rna.denovo.movers.RNA_Minimizer(rna_min_options)
```

In [152]:
### BEGIN SOLUTION
rna_min_options = core.import_pose.options.RNA_MinimizerOptions()
rna_min_options.set_max_iter(1000)
rna_minmizer = protocols.rna.denovo.movers.RNA_Minimizer(rna_min_options)
### END SOLUTION

Unlike in the case of using `MinMover`, things like an appropriate energy function and `MoveMap` are generated by default by the `RNA_Minimizer` object. By default, `RNA_Minimizer` uses the same high-resolution energy function as above (`stepwise/rna/rna_res_level_energy4.wts`).

All that remains is to apply it to the relevant pose.
```python
rna_minimizer.apply(frag_pose)
```

In [153]:
### BEGIN SOLUTION
rna_minmizer.apply(frag_pose)
### END SOLUTION

protocols.rna.denovo.movers.RNA_Minimizer: Orienting 2' hydroxyls...
protocols.rna.denovo.movers.RNA_Minimizer: Minimizing...round= 1
protocols.rna.denovo.movers.RNA_Minimizer: Orienting 2' hydroxyls...
protocols.rna.denovo.movers.RNA_Minimizer: Minimizing...round= 2
protocols.rna.denovo.movers.RNA_Minimizer: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       0.210    -205.344     -43.122
 fa_rep                       0.200      33.693       6.739
 fa_intra_rep                 0.003     170.729       0.495
 lk_nonpolar                  0.250      -4.344      -1.086
 fa_elec_rna_phos_phos        1.700      -0.780      -1.326
 rna_torsion                  1.000       6.365       6.365
 suiteness_bonus              1.000       0.000       0.000
 rna_sugar_close              0.820       0.833       0.683
 fa_stack                

Let's see what changes minimization has wrought on our structure:
```python
pmm.apply(frag_pose)
```

In [154]:
### BEGIN SOLUTION
pmm.apply(frag_pose)
### END SOLUTION

**Exercise**: Using the functions described in the first part of the notebook, report on the following with respect to our de novo folded sequence:
1. Which base pairs, if any, were recovered?
2. Which motifs, if any, were recovered?


TODO (for Matt): Fill in solution to above exercise, calling `classify_base_pairs` and `get_rna_motifs` as appropriate

## Additional Exercises

### Post-mortem

Examine our final folded structure and the hairpin from the first part of the tutorial and think about the following questions:
* How well did our brief structure prediction algorithm do at recovering the hairpin we examined at the beginning? 
* Which parts were more successfully recovered? Which parts less so? Why might this be? 
* What would you do to improve on this method as it stands? Feel free to implement any ideas you have.

### FAR + FAR = FARFAR

Write a function analogous to the `fragment_assembly` function above that 
* Accepts an input `Pose`
* Performs a round of minimzation using an `RNA_Minimizer`
* Returns the minimized structure

Using this new suboutine, craft your own `farfar` ( [Fragment Assembly of RNA with Full Atom Refinement](https://www.rosettacommons.org/docs/latest/application_documentation/rna/rna-denovo)) routine that performs multiple rounds of fragment assembly in a low-resolution potential followed by minimization in a high-resolution energy function. 

Try playing around with the various parameters and see how well you can recover the hairpin starting from just the sequence.
